* **Author**: Andrea Ziqing Gallardo Bendito

* **Project**: Bachelor Thesis - *Separación de fuentes musicales en conjuntos de cámara de música clásica*

* **GitHub Repo**: [MusicSourceSep](https://github.com/andrezg98/MusicSourceSep)

In this Notebook we are going to adapt our **augmented Bach10 dataset** to train the ***U-Net* model** implemented by the [Spleeter](https://research.deezer.com/projects/spleeter.html) library created by [Deezer](https://www.deezer.com/es/).



---



## **Project repository download and Library installations**



In [19]:
!pip install spleeter -q

     |████████████████████████████████| 50 kB 4.0 MB/s 
     |████████████████████████████████| 320.4 MB 47 kB/s 
     |████████████████████████████████| 10.5 MB 23.9 MB/s 
     |████████████████████████████████| 183 kB 76.2 MB/s 
     |████████████████████████████████| 65 kB 4.8 MB/s 
     |████████████████████████████████| 20.1 MB 1.2 MB/s 
     |████████████████████████████████| 459 kB 54.2 MB/s 
     |████████████████████████████████| 2.9 MB 75.3 MB/s 
     |████████████████████████████████| 55 kB 4.4 MB/s 
     |████████████████████████████████| 65 kB 4.1 MB/s 
     |████████████████████████████████| 54 kB 3.5 MB/s 
  Created wheel for librosa: filename=librosa-0.8.0-py3-none-any.whl size=201395 sha256=b97afb0a9def364e6208c5a10362a669576564d6fdd88b4c45597a78f8b05841
  Stored in directory: /root/.cache/pip/wheels/de/1e/aa/d91797ae7e1ce11853ee100bee9d1781ae9d750e7458c95afb
Successfully built librosa
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Unin

### **Imports**

In [24]:
# Imports
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split

# Duration
import wave
import contextlib

In [2]:
# To be able to write in bold format in the code
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

### **Project repository download**

In [ ]:
from getpass import getpass

# IMPORTANT: ¡Change when publishing the repository!

user = 'andrezg98'
password = getpass('GitHub repo password')
u = user; p = password; 
!git clone https://$u:$p@github.com/$u/MusicSourceSep.git
%cd MusicSourceSep/lib
!ls

del p, password

GitHub repo password··········
Cloning into 'MusicSourceSep'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 68 (delta 19), reused 21 (delta 4), pack-reused 0
Unpacking objects: 100% (68/68), done.
/content/drive/MyDrive/ING. SONIDO E IMAGEN/TFG/Preview Tests/Datasets/URMP-Dataset/MusicSourceSep/lib
feature_computation.py	urmp_dataset.py


## **Dataset Downloads**

* **Bach10_Augmented Dataset**
  
  Accessing the dataset stored in my Google Drive:

In [3]:
# Access to the Datasets folder
%cd /content/drive/MyDrive/ING. SONIDO E IMAGEN/TFG/Datasets

/content/drive/MyDrive/ING. SONIDO E IMAGEN/TFG/Datasets


> **Note**: To download the dataset for the first time, you can access [here](https://https://docs.google.com/forms/d/e/1FAIpQLSfJ1IdB7Ws2_m0wkkvS1hGm5GevGS3QmqBIoxiGDbw93yoPLQ/viewform?embedded=true&formkey=dGU3cmRlb1Q4RU5zTGNZeHUyRGFwaWc6MQ). The authors ask you to fill in a short form in order to keep track of the use of the dataset. Once downloaded, you can easily unzip and access it as follows. The dataset will be saved in the path you have specified.

> **Note 2**: The augmented dataset can be easily found in the link to the Drive shared folder of the GitHub repository.



---



##  **Prepare the source material for the model**
Let's prepare our augmented dataset to obtain the `Bach10_augmented_train.csv`, `Bach10_augmented_validation.csv` and `Bach10_test.csv` (without augmenting) needed to train and evaluate the ***U-Net* model**.

- **Bach10_Augmented**

In [5]:
# Declare the variables
AuSep = {'bassoon': list(), 'clarinet': list(), 
         'saxphone': list(), 'violin': list()}    # Separate audio list of each instrument
AuMix = list()                                    # Mix audio list
pieces = list()                                   # Name of the pieces
duration = list()                                 # Duration of the pieces
count = 0

# We go through the files in the folders and save them in each list
for dirpath, dirnames, filenames in os.walk("Bach10_Scaper/Bach10_Augmented"):
    if count == 0:
        for dirname in dirnames:
            pieces = dirnames
    if count > 0:
        for filename in filenames:
            filename = pieces[count-1] + "/" + filename
            if len(filename.split('-')) == 3:
                AuMix.append(filename)
                AuMix.sort(reverse=False)
                fname = 'Bach10_Scaper/Bach10_Augmented/' + filename
                with contextlib.closing(wave.open(fname,'r')) as f:
                    frames = f.getnframes()
                    rate = f.getframerate()
                    duration.append(frames/float(rate)) # seconds
                    duration.sort(reverse=False)
            else:
                if filename.split('-')[3] == 'bassoon.wav':
                    AuSep['bassoon'].append(filename)
                    AuSep['bassoon'].sort(reverse=False) 
                elif filename.split('-')[3] == 'clarinet.wav':
                    AuSep['clarinet'].append(filename)
                    AuSep['clarinet'].sort(reverse=False)
                elif filename.split('-')[3] == 'saxphone.wav':
                    AuSep['saxphone'].append(filename)
                    AuSep['saxphone'].sort(reverse=False)
                elif filename.split('-')[3] == 'violin.wav':
                    AuSep['violin'].append(filename)
                    AuSep['violin'].sort(reverse=False)
                else:
                    pass
    count += 1
    pieces.sort(reverse=False)

printmd("**Name of the pieces:**")
print(pieces)
printmd("**Separate Audio:**")
print(AuSep)
printmd("**Mix Audio:**")
print(AuMix)
printmd("**Duration:**")
print(duration)

**Name of the pieces:**

['01-AchGottundHerr_102', '01-AchGottundHerr_105', '01-AchGottundHerr_107', '01-AchGottundHerr_110', '01-AchGottundHerr_117', '01-AchGottundHerr_118', '01-AchGottundHerr_153', '01-AchGottundHerr_165', '01-AchGottundHerr_171', '01-AchGottundHerr_176', '01-AchGottundHerr_185', '01-AchGottundHerr_189', '01-AchGottundHerr_201', '01-AchGottundHerr_209', '01-AchGottundHerr_216', '01-AchGottundHerr_226', '01-AchGottundHerr_227', '01-AchGottundHerr_230', '01-AchGottundHerr_249', '01-AchGottundHerr_41', '01-AchGottundHerr_48', '01-AchGottundHerr_50', '01-AchGottundHerr_85', '02-AchLiebenChristen_101', '02-AchLiebenChristen_124', '02-AchLiebenChristen_132', '02-AchLiebenChristen_133', '02-AchLiebenChristen_149', '02-AchLiebenChristen_163', '02-AchLiebenChristen_167', '02-AchLiebenChristen_17', '02-AchLiebenChristen_177', '02-AchLiebenChristen_180', '02-AchLiebenChristen_186', '02-AchLiebenChristen_191', '02-AchLiebenChristen_194', '02-AchLiebenChristen_197', '02-AchLiebenChristen_204', '02-AchLi

**Separate Audio:**

{'bassoon': ['01-AchGottundHerr_102/01-AchGottundHerr_102-bassoon.wav', '01-AchGottundHerr_105/01-AchGottundHerr_105-bassoon.wav', '01-AchGottundHerr_107/01-AchGottundHerr_107-bassoon.wav', '01-AchGottundHerr_110/01-AchGottundHerr_110-bassoon.wav', '01-AchGottundHerr_117/01-AchGottundHerr_117-bassoon.wav', '01-AchGottundHerr_118/01-AchGottundHerr_118-bassoon.wav', '01-AchGottundHerr_153/01-AchGottundHerr_153-bassoon.wav', '01-AchGottundHerr_165/01-AchGottundHerr_165-bassoon.wav', '01-AchGottundHerr_171/01-AchGottundHerr_171-bassoon.wav', '01-AchGottundHerr_176/01-AchGottundHerr_176-bassoon.wav', '01-AchGottundHerr_185/01-AchGottundHerr_185-bassoon.wav', '01-AchGottundHerr_189/01-AchGottundHerr_189-bassoon.wav', '01-AchGottundHerr_201/01-AchGottundHerr_201-bassoon.wav', '01-AchGottundHerr_209/01-AchGottundHerr_209-bassoon.wav', '01-AchGottundHerr_216/01-AchGottundHerr_216-bassoon.wav', '01-AchGottundHerr_226/01-AchGottundHerr_226-bassoon.wav', '01-AchGottundHerr_227/01-AchGottundHerr_22

**Mix Audio:**

['01-AchGottundHerr_102/01-AchGottundHerr_102.wav', '01-AchGottundHerr_105/01-AchGottundHerr_105.wav', '01-AchGottundHerr_107/01-AchGottundHerr_107.wav', '01-AchGottundHerr_110/01-AchGottundHerr_110.wav', '01-AchGottundHerr_117/01-AchGottundHerr_117.wav', '01-AchGottundHerr_118/01-AchGottundHerr_118.wav', '01-AchGottundHerr_153/01-AchGottundHerr_153.wav', '01-AchGottundHerr_165/01-AchGottundHerr_165.wav', '01-AchGottundHerr_171/01-AchGottundHerr_171.wav', '01-AchGottundHerr_176/01-AchGottundHerr_176.wav', '01-AchGottundHerr_185/01-AchGottundHerr_185.wav', '01-AchGottundHerr_189/01-AchGottundHerr_189.wav', '01-AchGottundHerr_201/01-AchGottundHerr_201.wav', '01-AchGottundHerr_209/01-AchGottundHerr_209.wav', '01-AchGottundHerr_216/01-AchGottundHerr_216.wav', '01-AchGottundHerr_226/01-AchGottundHerr_226.wav', '01-AchGottundHerr_227/01-AchGottundHerr_227.wav', '01-AchGottundHerr_230/01-AchGottundHerr_230.wav', '01-AchGottundHerr_249/01-AchGottundHerr_249.wav', '01-AchGottundHerr_41/01-AchGo

**Duration:**

[25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.

* **Bach10**

In [7]:
# Declare the variables
AuSep_orig = {'bassoon': list(), 'clarinet': list(), 
              'saxphone': list(), 'violin': list()}    # Separate audio list of each instrument
AuMix_orig = list()                                    # Mix audio list
pieces_orig = list()                                   # Name of the pieces
duration_orig = list()                                 # Duration of the pieces
count = 0

# We go through the files in the folders and save them in each list
for dirpath, dirnames, filenames in os.walk("Bach10"):
    if count == 0:
        pieces_orig = dirnames
    if count > 0:
        for filename in filenames:
            filename = pieces_orig[count-1] + "/" + filename
            if filename.endswith('.wav'):
                if len(filename.split('-')) == 3:
                    AuMix_orig.append(filename)
                    AuMix_orig.sort(reverse=False)
                    fname = 'Bach10/' + filename
                    with contextlib.closing(wave.open(fname,'r')) as f:
                        frames = f.getnframes()
                        rate = f.getframerate()
                        duration_orig.append(frames/float(rate)) # seconds
                        duration_orig.sort(reverse=False)
                else:
                    if filename.split('-')[3] == 'bassoon.wav':
                        AuSep_orig['bassoon'].append(filename)
                        AuSep_orig['bassoon'].sort(reverse=False) 
                    elif filename.split('-')[3] == 'clarinet.wav':
                        AuSep_orig['clarinet'].append(filename)
                        AuSep_orig['clarinet'].sort(reverse=False)
                    elif filename.split('-')[3] == 'saxphone.wav':
                        AuSep_orig['saxphone'].append(filename)
                        AuSep_orig['saxphone'].sort(reverse=False)
                    elif filename.split('-')[3] == 'violin.wav':
                        AuSep_orig['violin'].append(filename)
                        AuSep_orig['violin'].sort(reverse=False)
                    else:
                        pass
            else:
                pass
    count += 1
    pieces_orig.sort(reverse=False)

printmd("**Name of the pieces:**")
print(pieces_orig)
printmd("**Separate Audio:**")
print(AuSep_orig)
printmd("**Mix Audio:**")
print(AuMix_orig)
printmd("**Duration:**")
print(duration_orig)

**Name of the pieces:**

['01-AchGottundHerr', '02-AchLiebenChristen', '03-ChristederdubistTagundLicht', '04-ChristeDuBeistand', '05-DieNacht', '06-DieSonne', '07-HerrGott', '08-FuerDeinenThron', '09-Jesus', '10-NunBitten', 'Code']


**Separate Audio:**

{'bassoon': ['01-AchGottundHerr/01-AchGottundHerr-bassoon.wav', '02-AchLiebenChristen/02-AchLiebenChristen-bassoon.wav', '03-ChristederdubistTagundLicht/03-ChristederdubistTagundLicht-bassoon.wav', '04-ChristeDuBeistand/04-ChristeDuBeistand-bassoon.wav', '05-DieNacht/05-DieNacht-bassoon.wav', '06-DieSonne/06-DieSonne-bassoon.wav', '07-HerrGott/07-HerrGott-bassoon.wav', '08-FuerDeinenThron/08-FuerDeinenThron-bassoon.wav', '09-Jesus/09-Jesus-bassoon.wav', '10-NunBitten/10-NunBitten-bassoon.wav'], 'clarinet': ['01-AchGottundHerr/01-AchGottundHerr-clarinet.wav', '02-AchLiebenChristen/02-AchLiebenChristen-clarinet.wav', '03-ChristederdubistTagundLicht/03-ChristederdubistTagundLicht-clarinet.wav', '04-ChristeDuBeistand/04-ChristeDuBeistand-clarinet.wav', '05-DieNacht/05-DieNacht-clarinet.wav', '06-DieSonne/06-DieSonne-clarinet.wav', '07-HerrGott/07-HerrGott-clarinet.wav', '08-FuerDeinenThron/08-FuerDeinenThron-clarinet.wav', '09-Jesus/09-Jesus-clarinet.wav', '10-NunBitten/10-NunBitten-clarin

**Mix Audio:**

['01-AchGottundHerr/01-AchGottundHerr.wav', '02-AchLiebenChristen/02-AchLiebenChristen.wav', '03-ChristederdubistTagundLicht/03-ChristederdubistTagundLicht.wav', '04-ChristeDuBeistand/04-ChristeDuBeistand.wav', '05-DieNacht/05-DieNacht.wav', '06-DieSonne/06-DieSonne.wav', '07-HerrGott/07-HerrGott.wav', '08-FuerDeinenThron/08-FuerDeinenThron.wav', '09-Jesus/09-Jesus.wav', '10-NunBitten/10-NunBitten.wav']


**Duration:**

[25.226643990929706, 25.363333333333333, 29.355374149659863, 32.566371882086166, 33.23877551020408, 33.28675736961451, 35.80780045351474, 37.39122448979592, 40.535873015873015, 41.66514739229025]


Now we join the new pieces generated with the original ones to obtain the whole dataset to train with.

In [10]:
AuMix = AuMix + AuMix_orig

AuSep['bassoon'] = AuSep['bassoon'] + AuSep_orig['bassoon']
AuSep['clarinet'] = AuSep['clarinet'] + AuSep_orig['clarinet']
AuSep['saxphone'] = AuSep['saxphone'] + AuSep_orig['saxphone']
AuSep['violin'] = AuSep['violin'] + AuSep_orig['violin']

AuSep = {'bassoon': AuSep['bassoon'], 'clarinet': AuSep['clarinet'], 'saxphone': AuSep['saxphone'], 'violin': AuSep['violin']}
duration = duration + duration_orig

printmd("**BACH10 AUGMENTED DATASET**")
print("----------------------------")
printmd("**Separate Audio:**")
print(AuSep)
printmd("**Mix Audio:**")
print(AuMix)
printmd("**Duration:**")
print(duration)

**BACH10 AUGMENTED DATASET**

----------------------------


**Separate Audio:**

{'bassoon': ['01-AchGottundHerr/01-AchGottundHerr-bassoon.wav', '02-AchLiebenChristen/02-AchLiebenChristen-bassoon.wav', '03-ChristederdubistTagundLicht/03-ChristederdubistTagundLicht-bassoon.wav', '04-ChristeDuBeistand/04-ChristeDuBeistand-bassoon.wav', '05-DieNacht/05-DieNacht-bassoon.wav', '06-DieSonne/06-DieSonne-bassoon.wav', '07-HerrGott/07-HerrGott-bassoon.wav', '08-FuerDeinenThron/08-FuerDeinenThron-bassoon.wav', '09-Jesus/09-Jesus-bassoon.wav', '10-NunBitten/10-NunBitten-bassoon.wav', '01-AchGottundHerr/01-AchGottundHerr-bassoon.wav', '02-AchLiebenChristen/02-AchLiebenChristen-bassoon.wav', '03-ChristederdubistTagundLicht/03-ChristederdubistTagundLicht-bassoon.wav', '04-ChristeDuBeistand/04-ChristeDuBeistand-bassoon.wav', '05-DieNacht/05-DieNacht-bassoon.wav', '06-DieSonne/06-DieSonne-bassoon.wav', '07-HerrGott/07-HerrGott-bassoon.wav', '08-FuerDeinenThron/08-FuerDeinenThron-bassoon.wav', '09-Jesus/09-Jesus-bassoon.wav', '10-NunBitten/10-NunBitten-bassoon.wav', '01-AchGottundH

**Mix Audio:**

['01-AchGottundHerr/01-AchGottundHerr.wav', '02-AchLiebenChristen/02-AchLiebenChristen.wav', '03-ChristederdubistTagundLicht/03-ChristederdubistTagundLicht.wav', '04-ChristeDuBeistand/04-ChristeDuBeistand.wav', '05-DieNacht/05-DieNacht.wav', '06-DieSonne/06-DieSonne.wav', '07-HerrGott/07-HerrGott.wav', '08-FuerDeinenThron/08-FuerDeinenThron.wav', '09-Jesus/09-Jesus.wav', '10-NunBitten/10-NunBitten.wav', '01-AchGottundHerr/01-AchGottundHerr.wav', '02-AchLiebenChristen/02-AchLiebenChristen.wav', '03-ChristederdubistTagundLicht/03-ChristederdubistTagundLicht.wav', '04-ChristeDuBeistand/04-ChristeDuBeistand.wav', '05-DieNacht/05-DieNacht.wav', '06-DieSonne/06-DieSonne.wav', '07-HerrGott/07-HerrGott.wav', '08-FuerDeinenThron/08-FuerDeinenThron.wav', '09-Jesus/09-Jesus.wav', '10-NunBitten/10-NunBitten.wav', '01-AchGottundHerr/01-AchGottundHerr.wav', '02-AchLiebenChristen/02-AchLiebenChristen.wav', '03-ChristederdubistTagundLicht/03-ChristederdubistTagundLicht.wav', '04-ChristeDuBeistand/04-C

**Duration:**

[25.226643990929706, 25.363333333333333, 29.355374149659863, 32.566371882086166, 33.23877551020408, 33.28675736961451, 35.80780045351474, 37.39122448979592, 40.535873015873015, 41.66514739229025, 25.226643990929706, 25.363333333333333, 29.355374149659863, 32.566371882086166, 33.23877551020408, 33.28675736961451, 35.80780045351474, 37.39122448979592, 40.535873015873015, 41.66514739229025, 25.226643990929706, 25.363333333333333, 29.355374149659863, 32.566371882086166, 33.23877551020408, 33.28675736961451, 35.80780045351474, 37.39122448979592, 40.535873015873015, 41.66514739229025]


After getting the required separated and mixed audio files, we can model the `.csv` files:

In [11]:
# DataFrame
df = pd.DataFrame(np.array([AuMix, AuSep['bassoon'], AuSep['clarinet'], AuSep['saxphone'], AuSep['violin'], duration]).T, columns=['mix_path', 'bassoon_path', 'clarinet_path', 'saxphone_path', 'violin_path', 'duration'])

First, divide our data set into a training and test set.

In [13]:
bach10_augmented_Train, bach10_test = train_test_split(df, test_size=0.1)

Next, we split our training set into training set and validation set.

In [14]:
bach10_augmented_train, bach10_augmented_validation = train_test_split(bach10_augmented_Train, test_size=0.1)

In [15]:
# Save the .csv files
bach10_augmented_train.to_csv(r'Bach10_spleeter/bach10_augmented_train.csv', index=False)
bach10_augmented_validation.to_csv(r'Bach10_spleeter/bach10_augmented_validation.csv', index=False)

In [16]:
bach10_augmented_train

,mix_path,bassoon_path,clarinet_path,saxphone_path,violin_path,duration
9,10-NunBitten/10-NunBitten.wav,10-NunBitten/10-NunBitten-bassoon.wav,10-NunBitten/10-NunBitten-clarinet.wav,10-NunBitten/10-NunBitten-saxphone.wav,10-NunBitten/10-NunBitten-violin.wav,41.66514739229025
0,01-AchGottundHerr/01-AchGottundHerr.wav,01-AchGottundHerr/01-AchGottundHerr-bassoon.wav,01-AchGottundHerr/01-AchGottundHerr-clarinet.wav,01-AchGottundHerr/01-AchGottundHerr-saxphone.wav,01-AchGottundHerr/01-AchGottundHerr-violin.wav,25.226643990929706
23,04-ChristeDuBeistand/04-ChristeDuBeistand.wav,04-ChristeDuBeistand/04-ChristeDuBeistand-bass...,04-ChristeDuBeistand/04-ChristeDuBeistand-clar...,04-ChristeDuBeistand/04-ChristeDuBeistand-saxp...,04-ChristeDuBeistand/04-ChristeDuBeistand-viol...,32.566371882086166
6,07-HerrGott/07-HerrGott.wav,07-HerrGott/07-HerrGott-bassoon.wav,07-HerrGott/07-HerrGott-clarinet.wav,07-HerrGott/07-HerrGott-saxphone.wav,07-HerrGott/07-HerrGott-violin.wav,35.80780045351474
19,10-NunBitten/10-NunBitten.wav,10-NunBitten/10-NunBitten-bassoon.wav,10-NunBitten/10-NunBitten-clarinet.wav,10-NunBitten/10-NunBitten-saxphone.wav,10-NunBitten/10-NunBitten-violin.wav,41.66514739229025
7,08-FuerDeinenThron/08-FuerDeinenThron.wav,08-FuerDeinenThron/08-FuerDeinenThron-bassoon.wav,08-FuerDeinenThron/08-FuerDeinenThron-clarinet...,08-FuerDeinenThron/08-FuerDeinenThron-saxphone...,08-FuerDeinenThron/08-FuerDeinenThron-violin.wav,37.39122448979592
12,03-ChristederdubistTagundLicht/03-Christederdu...,03-ChristederdubistTagundLicht/03-Christederdu...,03-ChristederdubistTagundLicht/03-Christederdu...,03-ChristederdubistTagundLicht/03-Christederdu...,03-ChristederdubistTagundLicht/03-Christederdu...,29.355374149659863
29,10-NunBitten/10-NunBitten.wav,10-NunBitten/10-NunBitten-bassoon.wav,10-NunBitten/10-NunBitten-clarinet.wav,10-NunBitten/10-NunBitten-saxphone.wav,10-NunBitten/10-NunBitten-violin.wav,41.66514739229025
21,02-AchLiebenChristen/02-AchLiebenChristen.wav,02-AchLiebenChristen/02-AchLiebenChristen-bass...,02-AchLiebenChristen/02-AchLiebenChristen-clar...,02-AchLiebenChristen/02-AchLiebenChristen-saxp...,02-AchLiebenChristen/02-AchLiebenChristen-viol...,25.363333333333333
14,05-DieNacht/05-DieNacht.wav,05-DieNacht/05-DieNacht-bassoon.wav,05-DieNacht/05-DieNacht-clarinet.wav,05-DieNacht/05-DieNacht-saxphone.wav,05-DieNacht/05-DieNacht-violin.wav,33.23877551020408


In [17]:
bach10_augmented_validation

,mix_path,bassoon_path,clarinet_path,saxphone_path,violin_path,duration
11,02-AchLiebenChristen/02-AchLiebenChristen.wav,02-AchLiebenChristen/02-AchLiebenChristen-bass...,02-AchLiebenChristen/02-AchLiebenChristen-clar...,02-AchLiebenChristen/02-AchLiebenChristen-saxp...,02-AchLiebenChristen/02-AchLiebenChristen-viol...,25.363333333333333
17,08-FuerDeinenThron/08-FuerDeinenThron.wav,08-FuerDeinenThron/08-FuerDeinenThron-bassoon.wav,08-FuerDeinenThron/08-FuerDeinenThron-clarinet...,08-FuerDeinenThron/08-FuerDeinenThron-saxphone...,08-FuerDeinenThron/08-FuerDeinenThron-violin.wav,37.39122448979592
26,07-HerrGott/07-HerrGott.wav,07-HerrGott/07-HerrGott-bassoon.wav,07-HerrGott/07-HerrGott-clarinet.wav,07-HerrGott/07-HerrGott-saxphone.wav,07-HerrGott/07-HerrGott-violin.wav,35.80780045351474




---



# **Training**

### ***U-Net Model***

To train the U-Net model implemented by the Spleeter library, it is necessary to change the training parameters in the `config.json` file that had to be created previously.

Example of `config.json`:

> ```
{
    "train_csv": "bach10_augmented_train.csv",
    "validation_csv": "bach10_augmented_validation.csv",
    "model_dir": "unet_model_base",
    "mix_name": "mix",
    "instrument_list": ["bassoon", "clarinet", "saxphone", "violin"],
    "sample_rate":44100,
    "frame_length":4096,
    "frame_step":1024,
    "T":512,
    "F":1024,
    "n_channels":2,
    "n_chunks_per_song":5,
    "separation_exponent":2,
    "mask_extension":"zeros",
    "learning_rate": 1e-4,
    "batch_size":4,
    "training_cache":"cache/training/",
    "validation_cache":"cache/validation/",
    "train_max_steps": 1500000,
    "throttle_secs":600,
    "random_seed":0,
    "save_checkpoints_steps":300,
    "save_summary_steps":5,
    "model":{
        "type":"unet.unet",
        "params":{
               "conv_activation":"ELU",
               "deconv_activation":"ELU"
        }
    }
}
```





> **Note**: Now, our train and validation csv are from the **Bach10_Augmented**.

> **Note 2**: change the name of the model if modified any parameter of `config.json`

In [22]:
# Model name: unet_model_base_augmented
!spleeter train --verbose -p "Bach10_spleeter/bach10_augmented_config.json" -d "Bach10_Scaper/Bach10_Augmented/"

INFO:tensorflow:Using config: {'_model_dir': 'unet_model_base_augmented', '_tf_random_seed': 0, '_save_summary_steps': 5, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.45
}
, '_keep_checkpoint_max': 2, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 10, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:spleeter:Start model training
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate

We move the new model (with the name defined in `config.json`) to the *models* folder of *Spleeter*.

In [23]:
!mv unet_model_base_augmented/ "Bach10_spleeter/models/"



---

